In [1]:
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
from glob import glob
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import geopandas as gpd
from shapely.geometry import Point
from einops import repeat
from copy import deepcopy
import rasterio

from sklearn.model_selection import KFold
from math import sqrt
SPINE_COLOR = 'gray'

In [2]:
def latexify(fig_width=None, fig_height=None, columns=1):
    """Set up matplotlib's RC params for LaTeX plotting.
    Call this before plotting a figure.

    Parameters
    ----------
    fig_width : float, optional, inches
    fig_height : float,  optional, inches
    columns : {1, 2}
    """

    # code adapted from http://www.scipy.org/Cookbook/Matplotlib/LaTeX_Examples

    # Width and max height in inches for IEEE journals taken from
    # computer.org/cms/Computer.org/Journal%20templates/transactions_art_guide.pdf

    assert(columns in [1,2])

    fig_width = 3.37
    fig_height = 1.5


    MAX_HEIGHT_INCHES = 8.0
    if fig_height > MAX_HEIGHT_INCHES:
        print("WARNING: fig_height too large:" + fig_height + 
              "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
        fig_height = MAX_HEIGHT_INCHES

    params = {'backend': 'ps',
            #   'text.latex.preamble': [r'\usepackage{gensymb}'],
              'axes.labelsize': 9, # fontsize for x and y labels (was 10)
              'axes.titlesize': 9,
              'font.size': 9, # was 10
              'legend.fontsize': 9, # was 10
              'xtick.labelsize': 9,
              'ytick.labelsize': 9,
              'text.usetex': True,
              'figure.figsize': [fig_width,fig_height],
              'font.family': 'serif'
    }

    matplotlib.rcParams.update(params)


def format_axes(ax):

    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)

    for spine in ['left', 'bottom']:
        ax.spines[spine].set_color(SPINE_COLOR)
        ax.spines[spine].set_linewidth(0.5)

    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_tick_params(direction='out', color=SPINE_COLOR)

    return ax

def format_axes(ax):

    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)

    for spine in ['left', 'bottom']:
        ax.spines[spine].set_color(SPINE_COLOR)
        ax.spines[spine].set_linewidth(0.5)

    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_tick_params(direction='out', color=SPINE_COLOR)

    return ax

In [3]:
with xr.open_dataset('/home/patel_zeel/aqmsp/aqmsp_data/datasets/cpcb/ijcai24/data.nc') as ds:
    pass
ds

<xarray.Dataset>
Dimensions:                    (time: 17520, station: 46)
Coordinates:
  * time                       (time) datetime64[ns] 2022-01-01T00:30:00 ... ...
  * station                    (station) object 'Alipur_Delhi_DPCC' ... 'Wazi...
    lat                        (station) float64 ...
    lon                        (station) float64 ...
    elevation                  (station) float64 ...
    pop_1km                    (station) float64 ...
    pop_2km                    (station) float64 ...
    pop_3km                    (station) float64 ...
    pop_4km                    (station) float64 ...
    pop_5km                    (station) float64 ...
    pop_10km                   (station) float64 ...
    d_motorway                 (station) float64 ...
Data variables: (12/417)
    TSURF_K                    (time, station) float32 ...
    SNOWEW_M                   (time, station) float32 ...
    SNOWAGE_HR                 (time, station) float32 ...
    PRATE_MMpH                 (time, station) float32 ...
    CLOUD_OD                   (time, station) float32 ...
    U10_MpS                    (time, station) float32 ...
    ...                         ...
    UWIND_MpS_LAY0_lag_1       (time, station) float32 ...
    UWIND_MpS_LAY0_lag_2       (time, station) float32 ...
    UWIND_MpS_LAY0_lag_3       (time, station) float32 ...
    UWIND_MpS_LAY0_lag_4       (time, station) float32 ...
    UWIND_MpS_LAY0_lag_5       (time, station) float32 ...
    UWIND_MpS_LAY0_lag_6       (time, station) float32 ...
Attributes: (12/33)
    IOAPI_VERSION:  $Id: @(#) ioapi library version 3.0 $                    ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    CDATE:          2023264
    CTIME:          73659
    WDATE:          2023264
    ...             ...
    VGLVLS:         [0. 0.]
    GDNAM:          ????????????????
    UPNAM:          CAMx2IOAPI      
    VAR-LIST:       TSURF_K         SNOWEW_M        SNOWAGE_HR      PRATE_MMp...
    FILEDESC:       I/O API formatted CAMx AVRG output                       ...
    HISTORY:

In [4]:
ds.station.values

array(['Alipur_Delhi_DPCC', 'Anand_Vihar_Delhi_DPCC',
       'Arya_Nagar_Bahadurgarh_HSPCB', 'Ashok_Vihar_Delhi_DPCC',
       'Aya_Nagar_Delhi_IMD', 'Bawana_Delhi_DPCC',
       'CRRI_Mathura_Road_Delhi_IMD',
       'Dr._Karni_Singh_Shooting_Range_Delhi_DPCC',
       'Dwarka_Sector_8_Delhi_DPCC_', 'IGI_Airport_(T3)_Delhi_IMD',
       'ITO_Delhi_CPCB', 'Indirapuram_Ghaziabad_UPPCB',
       'Jahangirpuri_Delhi_DPCC', 'Jawaharlal_Nehru_Stadium_Delhi_DPCC',
       'Knowledge_Park_III_Greater_Noida_UPPCB',
       'Knowledge_Park_V_Greater_Noida_UPPCB', 'Lodhi_Road_Delhi_IMD',
       'Major_Dhyan_Chand_National_Stadium_Delhi_DPCC',
       'Mandir_Marg_Delhi_DPCC', 'Mundka_Delhi_DPCC',
       'NSIT_Dwarka_Delhi_CPCB', 'Najafgarh_Delhi_DPCC',
       'Narela_Delhi_DPCC', 'Nathu_Colony_Ballabgarh_HSPCB',
       'Nehru_Nagar_Delhi_DPCC', 'North_Campus_DU_Delhi_IMD',
       'Okhla_Phase_2_Delhi_DPCC', 'Patparganj_Delhi_DPCC',
       'Punjabi_Bagh_Delhi_DPCC', 'Pusa_Delhi_DPCC',
       'R_K_Puram_De

In [5]:
delhi = gpd.read_file("/home/patel_zeel/aqmsp/aqmsp_data/datasets/shapefiles/delhi/Districts.shp")

latexify()
fig, ax = plt.subplots()
delhi.plot(ax=ax, color="none", edgecolor="k")
format_axes(ax)
# set axis off
ax.set_axis_off()
ax.scatter(77.328109, 28.625020, c='r', s=20, label="Ghazipur landfill", marker='x')

for sid, station in enumerate(["Indirapuram_Ghaziabad_UPPCB", "Sector_62_Noida_IMD", "Anand_Vihar_Delhi_DPCC"]):
    ax.scatter(ds.lon.sel(station=station).values, ds.lat.sel(station=station).values, label=station, marker='d', c=f'C{sid}', s=20)
ax.legend(bbox_to_anchor=(0.85, 1), loc='upper left')
plt.xlim(77.2, 77.4)
plt.ylim(28.5, 28.7)
plt.savefig("ghazipur_locs.pdf")
plt.show()

In [6]:
ghazipur_m28 = ds.sel(time=slice("2022-03-25", "2022-04-01"))

pol = "CO"
ghazipur_m28.sel(station="Indirapuram_Ghaziabad_UPPCB")[pol].plot(c="C0")
ghazipur_m28.sel(station="Anand_Vihar_Delhi_DPCC")[pol].plot(c="C1")
ghazipur_m28.sel(station="Sector_62_Noida_IMD")[pol].plot(c="C2")

plt.figure()
plt.vlines(ghazipur_m28.time.values[24*3-7], *plt.ylim(), color="k", linestyle="--", alpha=1, label="Fire starts")
# plt.vlines(ghazipur_m28.time.values[24*7], *plt.ylim(), color="k", linestyle="--", alpha=0.5)
plt.title("abc")
plt.xlabel("")
plt.legend();
plt.savefig("ghazipur_co.pdf")

In [7]:
plt.plot([0, 0], [1, 1])